<a href="https://colab.research.google.com/github/tcmb1987/ee259-project/blob/main/sionna_radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# IPython "magic function" for inline plots
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
import math

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Index of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

Number of GPUs available : 0


In [ ]:
array_size = 9
init_antenna_pos = [[0,0,0],[1,0,0],[2,0,0],[0,1,0],[1,1,0],[2,1,0],[0,2,0],[1,2,0],[2,2,0]] #9x3
#detect_obj = sionna.SceneObject()
ant = sionna.rt.Antenna("iso","V")
ant_array = sionna.rt.AntennaArray(ant,init_antenna_pos,True) # positions are trainable
init_tx_pos =[0,0,0]
radar_tx = sionna.rt.Transmitter("radar_tx",position=init_tx_pos,trainable_position=True,trainable_orientation=True)

init_obj_pos = [1000,1000,1000]
init_obj_or = [0,0,0]
# what is look at??
obj_tx = sionna.rt.Transmitter("obj_tx",position=init_obj_pos,orientation=init_obj_or,trainable_position=True,trainable_orientation=True)
obj_rx = sionna.rt.Receiver("obj_rx",position=init_obj_pos,orientation=init_obj_or,trainable_position=True,trainable_orientation=True)

# radar chirp (values from Radar lecture slides)
b = 12E12 # slew rate Hz/s
T = 45E-6 #45uS
fc = 77E9 #76GHz
t = np.linspace(0,T,1000)

# Transmit over noisy multi-path channel 
#no = sionna.utils.ebnodb2no(ebno_db, self.num_bits_per_symbol, self.coderate, self.rg)
       
# assume flat fading channel?
channel_1 = sionna.channel.RayleighBlockFading(num_rx=1,num_rx_ant=1,num_tx=1,num_tx_ant=array_size) # radar tx to object rx
channel_2 = sionna.channel.RayleighBlockFading(num_rx=1,num_rx_ant=array_size,num_tx = 1, num_tx_ant=1) # object tx to radar rx


#y = self.channel([x_rg, no]) 

In [ ]:
# send tx signal
no = 0.2 # Noise variance of the channel (arbitrarily chosen)
#s_tx = math.e**(1j*2*math.pi*(fc+b*t/2)*t)
# s_rx = alpha*s_tx(t-tau)*math.e**(1j*2*math.pi*fd*t)
num_bits_per_symbol = 1
batch_size = 1024 # number of symbols transmitted per antenna
init_tensor = np.array([1,-1])
radar_tx_tensor = tf.constant(tf.tile(init_tensor,multiples=[batch_size]))
print(radar_tx_tensor.shape)
# FIX MEEEE
#radar_tx_tensor_array = tf.repeat(radar_tx_tensor,repeats=array_size)
# figure out how to repeat
#radar_tx_tensor_array = tf.Tensor([radar_tx_tensor radar_tx_tensor radar_tx_tensor radar_tx_tensor radar_tx_tensor radar_tx_tensor radar_tx_tensor radar_tx_tensor radar_tx_tensor],shape=(batch_size,array_size))
#print(radar_tx_tensor_array.shape)

#pam_source = sionna.utils.PAMSource(num_bits_per_symbol)
#s_tx = pam_source([batch_size, array_size])
#obj_rx_signal,h_1 = channel_1([radar_tx_tensor,no])

(2048,)


In [ ]:
# test, not radar signal
num_tx_ant = 9
num_rx_ant = 1
num_bits_per_symbol = 4
batch_size = 1024
qam_source = sionna.utils.QAMSource(num_bits_per_symbol)
x = qam_source([batch_size, num_tx_ant])
#print(x.shape)
#print(x)
num_time_steps = 10*len(t) # num_time_steps = 10*length of radar pulse, arbitrarily chosen
filter_len = 1000 # what should i be????

channel_test = sionna.channel.RayleighBlockFading(num_rx=1,num_rx_ant=1,num_tx=1,num_tx_ant=9) # radar tx to object rx
h,tau=channel_test(batch_size,num_time_steps) 
time_channel = sionna.channel.ApplyTimeChannel(num_time_steps,filter_len)
time_channel_out = time_channel((x,h,no)) # what's wrong with me???

print(h.shape)
print(tau.shape)
print(h)
print(tau)

InvalidArgumentError: ignored

In [ ]:
a = tf.constant(radar_tx_tensor)
b = tf.Tensor()
b.shape


AttributeError: ignored

In [ ]:
a = tf.Tensor([[1.0,2.0] [3.0,4.0]],shape=(2,2))

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<ipython-input-59-9fbc00fd71b6>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  a = tf.Tensor([[1.0,2.0] [3.0,4.0]],shape=(2,2))


TypeError: ignored